# Junior Senior #1 - music detection
## Feature extraction

In [114]:
import os
import json

from pathlib import Path

import librosa
import pandas as pd
import numpy as np
import tensorflow as tf

In [115]:
data_root = "./data/source/"

dirs = os.listdir(data_root)

data_files = {
    class_: os.listdir(Path(data_root, class_)) for class_ in dirs
}

data_contents = {
    k: [] for k in data_files.keys()
}

In [ ]:
dataset = {
    "class": [],
    "filename": [],
    "chunk": [],
    "y": [],
    "sr": []
}

def split_by_seconds(dataset, root, file):
    class_ = root.split("/")[-1]
    y, sr = librosa.load(Path(root, file))
    
    n_seconds = int(y.shape[0] // sr)
    for i in range(n_seconds):
        y_sample = y[i * sr : (i+1) * sr]
        dataset["class"].append(1 if class_ == "muzyka" else 0)
        dataset["filename"].append(file)
        dataset["chunk"].append(i)
        dataset["y"].append(y_sample)
        dataset["sr"].append(sr)

for root, dirs, files in os.walk(data_root):
    for file in files:
        split_by_seconds(dataset, root, file)
        
df_dataset = pd.DataFrame(dataset)

In [117]:
def extract_mfcc(row):
    return list(librosa.feature.mfcc(y=row["y"], sr=row["sr"]).flatten())

def extract_rmse(row):
    return librosa.feature.rms(y=row["y"])

df_dataset["mfcc"] = df_dataset.apply(extract_mfcc, axis=1)
df_dataset["rms"] = df_dataset.apply(extract_rmse, axis=1)

In [126]:
df_dataset = df_dataset.sample(frac=1)

In [133]:
df_dataset[["class", "mfcc"]].to_json("data/processed/dataset.json", orient="records", double_precision=3)